In [2]:
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("crewai.ipynb")

logger.info("logger is set up and ready to rock")

INFO:crewai.ipynb:logger is set up and ready to rock


# Crew Ai Sandbox

## What is Crew AI?

CrewAi is a framework to make it easier to work with roleplaying AI (LLM) agents to achieve a tree of thought. It is open source and its source code may be found on [GitHub](https://github.com/crewAIInc/crewAI)

## Concepts involved

* LLM (Large Language Models)
* Prompt Engineering

## Setting up

I recommend using `virtualenv`, once you have it create a new `virtualenv` on this folder, active it (by `source`ing on Unix systems the `activate` script) and restore dependencies by running `pip install -r requirements.txt`.

You'll also want to have `ollama` running locally in order to pull and use local LLMs. In order to save money this sandbox uses local environments instead of expensive OpenAI calls. Ollama can be found at [their official website](https://ollama.com/download).

To make it easier to interact with ollama without doing manual steps in the terminal or gui we'll be using `ollama-python` to set things up using the notebook itself.

### Problems with Python 3.13

If you are using Python 3.13 you might run into issues (as of 2024-12-14) when installing CrewAi thru pip.

In order to get my environment to work I had to:

1. Set up my machine to run `rust` by using the script from [RustUp.rs](https://rustup.rs)
2. Add rust's cargo tools to my path by sourcing it on `.zshrc`
3. Exporting a specific variable in order to toggle compatibility between 3.12 and 3.13 when compiling: `export PYO3_USE_ABI3_FORWARD_COMPATIBILITY=1`

## Experimenting

### Installing the required libraries

As noted previously it's important to have a local copy of the requirements denoted in `requirements.txt`, if running only the notebook this setup phase may be achieved by running the cell below which will install the required dependencies.

I recommend still using at the very least a `venv` for this notebook not to mess up any system-wide dependencies you might have in your own environment.

In [3]:
# note: if you have already restored dependencies by looking at the requirements.txt file, you can skip this cell
# activating the environment is necessary to make sure the dependencies are installed in the right place
!source venv/bin/activate

# %pip runs pip directly from the notebook
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


### Ensuring that ollama is available in the environment

Before committing to a solution I would like to ensure that it is possible to have things set in a way that they are repeatable so, I'll begin this experiment by looking at the `ollama` library and if (using it) we can tell whenever ollama is or isn't running.

From looking at their docs seems like the simplest way to ensure ollama is available is to try and `list` models as, even if there are no models available, the result is non-error. That way if there is an exception it means that somehow `ollama` either isn't available or isn't set up properly.

In [4]:
logger.info("ensuring that ollama is up and running")
from ollama import list

try:
    list()
    logger.info("ollama is up and running")
except:
    logger.error("ollama is not running, exiting")
    exit(1)


INFO:crewai.ipynb:ensuring that ollama is up and running
INFO:httpx:HTTP Request: GET http://127.0.0.1:11434/api/tags "HTTP/1.1 200 OK"
INFO:crewai.ipynb:ollama is up and running


Once we know it's available we can go ahead and pull from one model we know it's available. As of the time of this experiment the model I'm most aware of is `llama3` so I'll use it as the desired one.

An extensive list of available models may be found on [ollama's website](https://ollama.com/library).

Once a model is pulled and available locally we can prompt it, without creating a chat, by using the `generate` function and passing in a prompt we`d like to ask it.

In [5]:
from ollama import pull, show, generate
DESIRED_MODEL = "llama3"

# attempt to show the model, if it doesn't exist it means we need to pull it
try:
    show(DESIRED_MODEL)
    logger.info(f"model {DESIRED_MODEL} found locally, no need to pull")
except:
    logger.info(f"model {DESIRED_MODEL} not found, pulling it")
    pull(DESIRED_MODEL)
    logger.info(f"model {DESIRED_MODEL} pulled successfully")

logger.info("asking the model why the sky is blue")
response = generate(DESIRED_MODEL, "why is the sky blue, in up to 256 characters")
logger.info(f"model's response: {response.response}")

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/show "HTTP/1.1 200 OK"
INFO:crewai.ipynb:model llama3 found locally, no need to pull
INFO:crewai.ipynb:asking the model why the sky is blue
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"
INFO:crewai.ipynb:model's response: The sky appears blue because of a phenomenon called Rayleigh scattering, where shorter (blue) wavelengths of light are scattered more than longer (red) wavelengths by tiny molecules of gases like nitrogen and oxygen in the atmosphere. This scattering effect gives the sky its blue hue!


### Crew AI - without Tools, simple process

By this point we have a working LLM locally that is able to be manually prompted and interacted with in a simple manner.

This is the basis in which CrewAI expands upon by providing a framework which can simulate a 'crew' composed by many agents (in this case AI ones) that interact with one another in order to achieve a task. They may also be handed out tools but we'll explore this later!

From [CrewAI's docs](https://docs.crewai.com/introduction#how-crewai-works) this is a process in which we:

0. Create a model - the LLM model which you'll be using for the crew
1. Create agents - the team members of your crew - that will be created upon a specific LLM Model with a specific role and a backstory
2. Create tasks - the work to be done - that will be assigned to specific agents in a crew
3. Create a crew - the team itself - by bringing in the agents, their tasks and a process they should follow to achieve an outcome

Linking this up to real world examples:

| CrewAI Concepts | Real life equivalent | Docs |
| --------------- | -------------------- | ---- |
| Agent | A software engineer in an Agile Squad | https://docs.crewai.com/concepts/agents |
| Task | A JIRA ticket an SE needs to work on | https://docs.crewai.com/concepts/tasks |
| Process | The development workflow they should follow | https://docs.crewai.com/concepts/processes |
| Crew | An agile squad composed of many members (Agents) of different roles (SE, SDET, UX) | https://docs.crewai.com/concepts/crews |

#### First problem: Create a landing page for a static website

As a software engineer I know how tempting it is to over-engineer something simple (such as this being a notebook rather than an one-off script lol) so I'll simulate a problem that would test if a LLM falls into such a trap if he/she is part of an Agile squad that happens to have a SE, an UX and a Business Owner.

My expected outcome is that by orchestrating those agents we may reach to:

1. requirements for the software engineer to work on, provided by a business owner
2. a simple solution, provided by the engineer, based on the owner's requirements
3. a high-level list of stories the engineer could work on based on, created by the business owner and the engineer together

Based on CrewAI docs I'll begin by creating the agents, then defining the tasks and finally assembling the crew.

For this cell we'll use the "verbose" mode which outputs the thoughts of each agent as they work on their tasks.

##### Business Owner: Karoline

Role: `Business Owner of a Candy Shop`

Backstory: 
```
Your name is Karoline and you are a small business owner of a candy shop called Karoline's Kandy, in Curitiba - Brazil, that sells candies such as Brigadeiros, Brownie and Cakes as well as gourmet coffee. You aren't keen on technology but knows that having your business show up on Social Medias and having its own link helps people to find it. You have a limited budget for this project because you just rented a venue close to a huge avenue where you expect both locals and tourists to pass by your store. Right now you are very reluctant to pursue virtual sales due to the complicated logistics required to support it.
```

Goal: `have a simple landing page for my business and an idea of how much, and long, it would take to build and maintain`

##### Software Engineer: Jakan

Role: `Consultant Software Engineer`

Backstory:
```
Your name is Jakan and are a software engineer working solo, born and raised in São Paulo - Brazil, you take on freelance opportunities by connecting with people over LinkedIn and word-of-mouth. You have worked on the IT field for over 10 years now, has deployed many systems by now and is expert on backend and frontend development. You are also able to quickly research non-technical information in order to make informed decisions and to support your customers on things that aren't related to code. You take a pragmatic and DevOps approach to problems, creating code that is refactorable but isn't 'perfect' on the first pass. Your main technologies are .NET 6 using the C# language and Vue with Typescript. Your preferred cloud provider is Azure.
```

Goal: `solve problems by using technology in a way that the results are reliable yet fit within a business' budget limits`

##### Task: Define the identify of your Business 

Must be done solo.

Description:
```
Define the branding of your business, taking into account your favorite colors (Red and White), your target audience (middle class and tourists) and a limited budget. Write a report defining what colors you'd like used when referring to your brand, its logo, social media you'd like to be on, digital experiences you'd like to provide your customers and the tone you'd like to be used in your brand's communication. Be conscious of your limited budget, keep it simple.
```

##### Task: Create a system design for Karoline's landing page

Must be done together.

Description:
```
Work out with other stakeholders to define a system design that meets their needs and fits their budgets. Take into account all the steps required to deploy a system to production and making it accessible in a secure way on customer-grade browsers. Provide a report on the high-level system design you would create to solve the problem at hand, focus on artifacts's names, their purpose and interactions in a high level.
```

##### Task: Agree upon a list of stories that should be part of a Minimum Viable Product

Must be done together.

Description:
```
Given the business needs and a system design work together to highlight a simple minimum viable product that could be delivered in less than a month to quickly provide value to stakeholders. Provide a report listing out User Stories (and its Sub-Tasks, if complex) and Tasks (non-code work) that need to be done in order to achieve that MVP state.
```

In [5]:
from crewai import Agent, Task, Process, Crew, LLM

# creating an Ollama based agent
ollamaAgent = LLM(model=f"ollama/{DESIRED_MODEL}", temperature=0.4)

logger.info("defining agents")

businessOwner = Agent(
    role="business owner of Karoline's Kandy",
    goal="have a simple landing page for my business and an idea of how much, and long, it would take to build and maintain",
    backstory="""
    Your name is Karoline and you are a small business owner of a candy shop called Karoline's Kandy, in Curitiba - Brazil, that sells candies such as Brigadeiros, Brownie and Cakes as well as gourmet coffee. 
    You aren't keen on technology but knows that having your business show up on Social Medias and having its own link helps people to find it. 
    You have a limited budget for this project because you just rented a venue close to a huge avenue where you expect both locals and tourists to pass by your store. 
    Right now you are very reluctant to pursue virtual sales due to the complicated logistics required to support it""",
    verbose=True,
    allow_delegation=False,
    llm=ollamaAgent
)

softwareEngineer = Agent(
    role="consulting software engineer",
    goal="solve problems by using technology in a way that the results are reliable yet fit within a business' budget limits",
    backstory="""
    Your name is Jakan and are a software engineer working solo, born and raised in São Paulo - Brazil, you take on freelance opportunities by connecting with people over LinkedIn and word-of-mouth. 
    You have worked on the IT field for over 10 years now, has deployed many systems by now and is expert on backend and frontend development. 
    You are also able to quickly research non-technical information in order to make informed decisions and to support your customers on things that aren't related to code. 
    You take a pragmatic and DevOps approach to problems, creating code that is refactorable but isn't 'perfect' on the first pass. 
    Your main technologies are .NET 6 using the C# language and Vue with Typescript. 
    Your preferred cloud provider is Azure.""",
    verbose=True,
    allow_delegation=False,
    llm=ollamaAgent
)

logger.info("defining tasks")
defineIdentity = Task(
    name="Define identity",
    description="""
    Define the branding of your business, taking into account your favorite colors (Red and White), your target audience (middle class and tourists) and a limited budget. 
    Be conscious of your limited budget, keep it simple.
    """,
    expected_output="Write a report defining what colors you'd like used when referring to your brand, its logo, social media you'd like to be on, digital experiences you'd like to provide your customers and the tone you'd like to be used in your brand's communication.",
    agent=businessOwner
)

defineSystemDesign = Task(
    name="Define system design",
    description="""
    Work out with other stakeholders to define a system design that meets their needs and fits their budgets.
    When in doubt if you should make something simple or complex consult with your stakeholder and their budget and needs.
    Take into account all the steps required to deploy a system to production and making it accessible in a secure way on customer-grade browsers. 
    """,
    expected_output="Provide a report, in a markdown format, of the high-level system design you would create to solve the problem at hand, focus on artifacts's names, their purpose and interactions in a high level.",
    agent=softwareEngineer,
    context=[defineIdentity]
)

defineMvp = Task(
    name="Define MVP",
    description="""
    Given the business needs and the system design work together to highlight a simple minimum viable product that could be delivered in less than a month to quickly provide value to stakeholders.
    """,
    agent=softwareEngineer,
    expected_output="Provide a report, in a markdown format, listing out User Stories (and its Sub-Tasks, if complex) and Tasks (non-code work) that need to be done in order to achieve that MVP state. Make sure you include acceptance criteria and a high-level step-by-step one should follow to complete it.",
    context=[defineSystemDesign, defineIdentity],
)

logger.info("defining the crew")
dynamicDuo = Crew(
    agents=[businessOwner, softwareEngineer],
    tasks=[defineIdentity, defineSystemDesign, defineMvp],
    verbose=True,
    process=Process.sequential
)

logger.info("kicking off the crew")
result = dynamicDuo.kickoff()

logger.info("crew has finished")
logger.info(f"result: {result}")

INFO:httpx:HTTP Request: GET https://raw.githubusercontent.com/BerriAI/litellm/main/model_prices_and_context_window.json "HTTP/1.1 200 OK"
INFO:crewai.ipynb:defining agents
INFO:crewai.ipynb:defining tasks
INFO:crewai.ipynb:defining the crew
INFO:crewai.ipynb:kicking off the crew
20:09:58 - LiteLLM:INFO: utils.py:2667 - 
LiteLLM completion() model= llama3; provider = ollama
INFO:LiteLLM:
LiteLLM completion() model= llama3; provider = ollama


# Agent: business owner of Karoline's Kandy
## Task: 
    Define the branding of your business, taking into account your favorite colors (Red and White), your target audience (middle class and tourists) and a limited budget. 
    Be conscious of your limited budget, keep it simple.
    


20:10:18 - LiteLLM:INFO: utils.py:905 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
20:10:18 - LiteLLM:INFO: utils.py:2667 - 
LiteLLM completion() model= llama3; provider = ollama
INFO:LiteLLM:
LiteLLM completion() model= llama3; provider = ollama
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"




# Agent: business owner of Karoline's Kandy
## Final Answer: 
**Branding Report for Karoline's Kandy**

As the owner of Karoline's Kandy, my goal is to create a strong brand identity that reflects my business's values and appeals to my target audience. Given my limited budget, I will focus on simplicity and effectiveness.

**Colors:**
I would like to use Red (#FF0000) and White (#FFFFFF) as the primary colors for my brand. These colors are bold, eye-catching, and evoke a sense of excitement and energy, which aligns with the experience of indulging in delicious candies and gourmet coffee at Karoline's Kandy.

**Logo:**
The logo will feature a stylized letter "K" made up of candy shapes (e.g., Brigadeiros or Brownies) in Red, surrounded by a White circle. The logo will be simple, yet distinctive and memorable.

**Social Media:**
To reach my target audience, I would like to establish a presence on the following social media platforms:

1. Facebook: to share updates, promotions, and behi

20:10:53 - LiteLLM:INFO: utils.py:905 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
20:10:53 - LiteLLM:INFO: utils.py:2667 - 
LiteLLM completion() model= llama3; provider = ollama
INFO:LiteLLM:
LiteLLM completion() model= llama3; provider = ollama
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"




# Agent: consulting software engineer
## Final Answer: 
**Karoline's Kandy Branding System Design**

### Overview
The Karoline's Kandy branding system will be designed to reflect the business's values, appeal to its target audience, and provide a seamless digital experience. The system will consist of five main components: Logo, Social Media, Digital Experiences, Tone, and Budget Allocation.

### Artifacts
#### Logo
* Purpose: Represent Karoline's Kandy brand identity
* Interactions:
	+ Will be used on the website, social media profiles, marketing materials, and other digital platforms
	+ Will be designed to be simple, yet distinctive and memorable

#### Social Media
* Purpose: Establish a presence on Facebook, Instagram, Twitter, and other relevant platforms
* Interactions:
	+ Will be used for sharing updates, promotions, behind-the-scenes glimpses of the candy-making process, visually appealing images of products, customer experiences, and events at Karoline's Kandy
	+ Will be mana

20:11:28 - LiteLLM:INFO: utils.py:905 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:crewai.ipynb:crew has finished
INFO:crewai.ipynb:result: **MVP Report for Karoline's Kandy Branding System**

### User Stories

#### Logo Design
As a customer, I want to see a logo that represents Karoline's Kandy brand identity, so that I can recognize and remember the brand.

* Sub-Tasks:
	+ Create a logo design concept based on the provided requirements (Red and White colors)
	+ Review and refine the design concept with Karoline's Kandy stakeholders
	+ Finalize the logo design and obtain approval from Karoline's Kandy

#### Social Media Presence
As a customer, I want to see Karoline's Kandy on social media platforms, so that I can stay updated on promotions, events, and be



# Agent: consulting software engineer
## Final Answer: 
**MVP Report for Karoline's Kandy Branding System**

### User Stories

#### Logo Design
As a customer, I want to see a logo that represents Karoline's Kandy brand identity, so that I can recognize and remember the brand.

* Sub-Tasks:
	+ Create a logo design concept based on the provided requirements (Red and White colors)
	+ Review and refine the design concept with Karoline's Kandy stakeholders
	+ Finalize the logo design and obtain approval from Karoline's Kandy

#### Social Media Presence
As a customer, I want to see Karoline's Kandy on social media platforms, so that I can stay updated on promotions, events, and behind-the-scenes glimpses of the candy-making process.

* Sub-Tasks:
	+ Set up social media profiles on Facebook, Instagram, Twitter, and other relevant platforms
	+ Create a content calendar for regular posting and engagement

#### Website Development
As a customer, I want to access basic information about Karolin

INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
ERROR:opentelemetry.sdk.trace.export:Exception while exporting Span batch.
Traceback (most recent call last):
  File "/Users/ralves/Projects/Personal/python-sandbox/crew-ai/venv/lib/python3.13/site-packages/urllib3/connectionpool.py", line 536, in _make_request
    response = conn.getresponse()
  File "/Users/ralves/Projects/Personal/python-sandbox/crew-ai/venv/lib/python3.13/site-packages/urllib3/connection.py", line 507, in getresponse
    httplib_response = super().getresponse()
  File "/opt/homebrew/Cellar/python@3.13/3.13.0_1/Frameworks/Python.framework/Versions/3.13/lib/python3.13/http/client.py", line 1428, in getresponse
    response.begin()
    ~~~~~~~~~~~~~~^^
  File "/opt/homebrew/Cellar/python@3.13/3.13.0_1/Frameworks/Python.framework/Versions/3.13/lib/python3.13/http/client.py", line 331, in begin
    version, status, reason = self._read_status()
                              ~~~~~~~~~~~~~~~~~^

##### Results from Llama3

Here's the final output from the collaboration between the agents:

```markdown
**MVP Report for Karoline's Kandy Branding System**

### User Stories

#### Logo Design
As a customer, I want to see a logo that represents Karoline's Kandy brand identity, so that I can recognize and remember the brand.

* Sub-Tasks:
	+ Create a logo design concept based on the provided requirements (Red and White colors)
	+ Review and refine the design concept with Karoline's Kandy stakeholders
	+ Finalize the logo design and obtain approval from Karoline's Kandy

#### Social Media Presence
As a customer, I want to see Karoline's Kandy on social media platforms, so that I can stay updated on promotions, events, and behind-the-scenes glimpses of the candy-making process.

* Sub-Tasks:
	+ Set up social media profiles on Facebook, Instagram, Twitter, and other relevant platforms
	+ Create a content calendar for regular posting and engagement

#### Website Development
As a customer, I want to access basic information about Karoline's Kandy products, store hours, and contact details, so that I can plan my visit or make informed purchasing decisions.

* Sub-Tasks:
	+ Design and develop a simple, easy-to-navigate website with basic information about products, store hours, and contact details
	+ Implement a content management system for future updates

#### Email Newsletter System
As a customer, I want to receive regular email newsletters from Karoline's Kandy, so that I can stay informed about new products, promotions, and events.

* Sub-Tasks:
	+ Design and develop an email newsletter system with subscription management
	+ Create a content calendar for regular newsletter sending

### Tasks (non-code work)

#### Branding Report
Create a comprehensive branding report outlining the brand's identity, values, and communication strategy. This will serve as the foundation for all future marketing efforts.

* Sub-Tasks:
	+ Research and gather information about Karoline's Kandy's target audience, competitors, and industry trends
	+ Develop a brand positioning statement and messaging framework
	+ Create a visual identity guide (color palette, typography, imagery) for the brand

#### Social Media Content Calendar
Create a content calendar for social media posting and engagement. This will ensure consistent and regular posting across all platforms.

* Sub-Tasks:
	+ Research and gather information about Karoline's Kandy's products, services, and events
	+ Develop a content strategy and calendar for each social media platform
	+ Create engaging content (images, videos, text) for regular posting

#### Website Content Creation
Create website content that accurately reflects Karoline's Kandy's brand identity and values. This will include product information, store hours, contact details, and other relevant information.

* Sub-Tasks:
	+ Research and gather information about Karoline's Kandy's products, services, and events
	+ Develop a content strategy for the website
	+ Create engaging website content (text, images, videos) that aligns with the brand's identity and values

By completing these user stories and tasks, we can establish a strong foundation for Karoline's Kandy's branding system and digital presence.
```

So, to conclude this piece of the experiment:

* Asking question between agents (aka the `allow_delegation` parameter) fails with some weird `dict` errors
* The agents successfully complete each of their assigned Tasks
* Tasks successfully take into account the context of previous tasks
* Agents are mindful about the expected outcome and what should be achieved for each task.

##### Repeating the experiment but using a different model

From reading the [CrewAi Subreddit](https://www.reddit.com/r/crewai/) there are many reports saying that different models (specially the Ollama ones) have different results even if using the same amount of parameters.

To test this out let's repeat the experiment by changing the agent's LLM model to `mistral` and seeing how the output changes.

In [11]:
MISTRAL_MODEL = "mistral"
# ensure ollama has mistral available
try:
    show(MISTRAL_MODEL)
    logger.info(f"model {MISTRAL_MODEL} found locally, no need to pull")
except:
    logger.info(f"model {MISTRAL_MODEL} not found, pulling it")
    pull(MISTRAL_MODEL)
    logger.info(f"model {MISTRAL_MODEL} pulled successfully")

# creating an Ollama based agent
mistral_llm = LLM(model=f"ollama/{MISTRAL_MODEL}", temperature=0.4)

logger.info("changing agents from %s over to %s", DESIRED_MODEL, MISTRAL_MODEL)
businessOwner.llm = mistral_llm
softwareEngineer.llm = mistral_llm

logger.info("kicking off the crew again")
result = dynamicDuo.kickoff()

logger.info("crew has finished")
logger.info(f"result: {result}")

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/show "HTTP/1.1 404 Not Found"
INFO:crewai.ipynb:model gemma2:9b not found, pulling it
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/pull "HTTP/1.1 200 OK"
INFO:crewai.ipynb:model gemma2:9b pulled successfully
INFO:crewai.ipynb:changing agents from llama3 over to gemma2:9b
INFO:crewai.ipynb:kicking off the crew again
20:49:06 - LiteLLM:INFO: utils.py:2667 - 
LiteLLM completion() model= gemma2:9b; provider = ollama
INFO:LiteLLM:
LiteLLM completion() model= gemma2:9b; provider = ollama


# Agent: business owner of Karoline's Kandy
## Task: 
    Define the branding of your business, taking into account your favorite colors (Red and White), your target audience (middle class and tourists) and a limited budget. 
    Be conscious of your limited budget, keep it simple.
    


20:49:17 - LiteLLM:INFO: utils.py:905 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
20:49:17 - LiteLLM:INFO: utils.py:2667 - 
LiteLLM completion() model= gemma2:9b; provider = ollama
INFO:LiteLLM:
LiteLLM completion() model= gemma2:9b; provider = ollama
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"


# Agent: consulting software engineer
## Task: What are some simple branding strategies that work well for small businesses with limited budgets?


20:49:43 - LiteLLM:INFO: utils.py:905 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
20:49:43 - LiteLLM:INFO: utils.py:2667 - 
LiteLLM completion() model= gemma2:9b; provider = ollama
INFO:LiteLLM:
LiteLLM completion() model= gemma2:9b; provider = ollama
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"


...

20:52:44 - LiteLLM:INFO: utils.py:905 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
20:52:44 - LiteLLM:INFO: utils.py:2667 - 
LiteLLM completion() model= gemma2:9b; provider = ollama
INFO:LiteLLM:
LiteLLM completion() model= gemma2:9b; provider = ollama
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"




# Agent: business owner of Karoline's Kandy
## Final Answer: 
For Karoline's Kandy's MVP website, we need these essential pages to attract customers and drive foot traffic:

**1. Homepage:**

*   **Headline:** "Karoline's Kandy - Sweeten Your Day!" (Catchy and inviting)
*   **Hero Image:** A delicious-looking photo of Karoline's signature Brigadeiros or Brownie, with a steaming cup of gourmet coffee in the background. 
*   **Short Description:**  "Experience the taste of Brazil at Karoline's Kandy! We offer homemade Brigadeiros, decadent Brownies, and freshly baked Cakes alongside our rich, aromatic gourmet coffee. Come visit us for a sweet treat!" 
*   **Call to Action Button:** "Visit Us Today!" (Links to Google Maps location)

**2. Menu Page:**

*   **Clear Categories:**  Brigadeiros, Brownies, Cakes, Coffee
*   **High-Quality Images:** Mouthwatering photos of each item on the menu.
*   **Descriptive Text:**  Brief but enticing descriptions of each treat, highlighting key ingredie

20:53:22 - LiteLLM:INFO: utils.py:905 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
20:53:22 - LiteLLM:INFO: utils.py:2667 - 
LiteLLM completion() model= gemma2:9b; provider = ollama
INFO:LiteLLM:
LiteLLM completion() model= gemma2:9b; provider = ollama
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
20:54:02 - LiteLLM:INFO: utils.py:905 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"




# Agent: consulting software engineer
## Final Answer: 
## Karoline's Kandy MVP Website - User Stories and Content Outline

**Goal:**  Create a simple, user-friendly website that highlights Karoline's Kandy's most popular products, encourages customer visits, and captures contact information.

**Target Audience:** Locals in Curitiba looking for sweet treats, tourists seeking authentic Brazilian desserts, coffee lovers.

**User Stories:**

* **As a local resident,** I want to easily find the location and hours of Karoline's Kandy so I can stop by for a delicious treat after work.
* **As a tourist,** I want to see photos and descriptions of popular Brazilian desserts like Brigadeiros and Brownies to decide what to try. 
* **As a coffee lover,** I want to know about the quality and origin of Karoline's Kandy's coffee beans and brewing methods.
* **As a potential customer,** I want to be able to contact Karoline's Kandy with questions or feedback easily through a simple form or email add

INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"


The result that we get from `mistral` backed agents is:

```markdown
## Minimum Viable Product (MVP) for Karoline's Kandy Branding Report

### User Stories

#### 1. User Story - Frontend Application Development

**Acceptance Criteria:**

- As a user, I want to visit the website and view the homepage, product listings, and contact information.
- As a user, I want to be able to submit a contact form with my name, email address, and message.
- As a user, I want to view detailed product information, including images, descriptions, and prices.

##### Sub-Tasks:

1.1 Design the homepage layout and navigation menu.
1.2 Create product listing pages with filtering options (e.g., category, price range).
1.3 Develop a contact form with validation and submission functionality.
1.4 Implement responsive design for mobile devices.

#### 2. User Story - Backend Application Development

**Acceptance Criteria:**

- As a user, I want the website to load quickly and reliably.
- As a user, I want the contact form submissions to be processed and sent as an email.
- As a developer, I want to be able to manage product data, including images, descriptions, and prices.

##### Sub-Tasks:

2.1 Design RESTful APIs for frontend communication.
2.2 Implement data validation and error handling in the backend.
2.3 Create database tables for products, categories, and user submissions.
2.4 Implement email functionality to send contact form submissions.

#### 3. User Story - Database Setup and Deployment

**Acceptance Criteria:**

- As a developer, I want to set up the database on Azure SQL Server and configure it for optimal performance.
- As a developer, I want to ensure secure access to the database using Azure Active Directory.
- As a developer, I want to deploy the frontend and backend applications to Azure App Service and Web App Service (or Azure Functions), respectively.

##### Sub-Tasks:

3.1 Set up the database schema for products, categories, and user submissions.
3.2 Configure secure access to the database using Azure Active Directory.
3.3 Deploy the frontend application to an Azure App Service.
3.4 Deploy the backend application to an Azure Web App Service (or Azure Functions).

### Next Steps

- Test the MVP locally and on various devices to ensure compatibility.
- Implement SSL for secure communication between the client and server.
- Set up email notifications for new contact form submissions.
- Design and implement social media integrations (Facebook, Instagram, Google My Business).
- Develop an email newsletter system for regular updates and promotions.
- Continuously monitor and improve the application's performance and user experience based on feedback and analytics data.
```

Unfortunately `mistral` is still unable to use the `allow_delegation` parameter to ask questions to its peers.

To conclude the first problem:

* `mistral` seems to be more direct to the point
* `llama` generates a longer output
* looking at the expected outcome (a MVP for a simple static page) both models overengineered but the solution and stories provided by `mistral` were less overkill than the ones from `llama3`

###### Trying out a LLM that's able to collaborate

After some more googling around I found out that `gemma2` with `9bi` parameters seems to be a decent llm for CrewAi when doing local development so let`s try this one out with the same tasks for the sake of completion and compare its final result to the ones provided by mistral and llama3.


In [15]:
GOOGLE_MODEL = "gemma2:9b"
# ensure ollama has mistral available
try:
    show(GOOGLE_MODEL)
    logger.info(f"model {GOOGLE_MODEL} found locally, no need to pull")
except:
    logger.info(f"model {GOOGLE_MODEL} not found, pulling it")
    pull(GOOGLE_MODEL)
    logger.info(f"model {GOOGLE_MODEL} pulled successfully")

# creating an Ollama based agent
gemma2_llm = LLM(model=f"ollama/{GOOGLE_MODEL}", temperature=0.4)

logger.info("changing agents from %s over to %s", MISTRAL_MODEL, GOOGLE_MODEL)
businessOwner.llm = gemma2_llm
softwareEngineer.llm = gemma2_llm

logger.info("changing agents off of verbose mode")
businessOwner.verbose = False
softwareEngineer.verbose = False

logger.info("kicking off the crew again, third time is the charm!")
result = dynamicDuo.kickoff()

logger.info("crew has finished")
logger.info(f"result: {result}")

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/show "HTTP/1.1 200 OK"
INFO:crewai.ipynb:model gemma2:9b found locally, no need to pull
INFO:crewai.ipynb:changing agents from gemma2:9b over to gemma2:9b
INFO:crewai.ipynb:changing agents off of verbose mode
INFO:crewai.ipynb:kicking off the crew again, third time is the charm!
11:54:42 - LiteLLM:INFO: utils.py:2667 - 
LiteLLM completion() model= gemma2:9b; provider = ollama
INFO:LiteLLM:
LiteLLM completion() model= gemma2:9b; provider = ollama


# Agent: business owner of Karoline's Kandy
## Task: 
    Define the branding of your business, taking into account your favorite colors (Red and White), your target audience (middle class and tourists) and a limited budget. 
    Be conscious of your limited budget, keep it simple.
    


11:54:55 - LiteLLM:INFO: utils.py:905 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
11:54:55 - LiteLLM:INFO: utils.py:2667 - 
LiteLLM completion() model= gemma2:9b; provider = ollama
INFO:LiteLLM:
LiteLLM completion() model= gemma2:9b; provider = ollama
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"


# Agent: consulting software engineer
## Task: What color combinations appeal to middle-class consumers and tourists in Brazil?


11:55:18 - LiteLLM:INFO: utils.py:905 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
11:55:18 - LiteLLM:INFO: utils.py:2667 - 
LiteLLM completion() model= gemma2:9b; provider = ollama
INFO:LiteLLM:
LiteLLM completion() model= gemma2:9b; provider = ollama
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"


...

12:01:12 - LiteLLM:INFO: utils.py:905 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
12:01:12 - LiteLLM:INFO: utils.py:2667 - 
LiteLLM completion() model= gemma2:9b; provider = ollama
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:LiteLLM:
LiteLLM completion() model= gemma2:9b; provider = ollama
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"




# Agent: business owner of Karoline's Kandy
## Final Answer: 
Well, since I'm not big on technology, the system needs to be simple and easy to understand.  Here are some things that would be really helpful:

* **Easy Tracking:** I need to know how many of each candy and cake I have at all times. The system should let me quickly see what's running low so I can order more before I run out. 
* **Sales Reporting:** It would be great to see which candies and cakes are selling the best. That way, I can focus on making more of those popular items and maybe even create new ones based on what people like.  I could also use this to plan promotions for slower-selling items.
* **Ingredient Tracking:**  A lot of my recipes use similar ingredients, so it would be helpful to keep track of how much of each ingredient I have on hand. This way, I can avoid running out of something important in the middle of making a big batch of Brigadeiros or Brownie!
* **Expiration Dates:** Some candies and cakes ha

12:01:31 - LiteLLM:INFO: utils.py:905 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:crewai.ipynb:crew has finished
INFO:crewai.ipynb:result: The MVP for Karoline's Kandy's inventory management system should focus on these core features:

1. **Simple Inventory Tracking:**  Allowing easy input and viewing of current stock levels for each candy, cake, and key ingredient. 
2. **Sales Reporting:** Generate reports showing which items are selling well and which are slower movers to inform production and promotional decisions.
3. **Ingredient Management:** Track inventory levels for common ingredients used in multiple recipes to prevent shortages.
4. **Expiration Date Reminders:**  Set expiration dates for perishable items and send alerts when they are approaching 



# Agent: consulting software engineer
## Final Answer: 
The MVP for Karoline's Kandy's inventory management system should focus on these core features:

1. **Simple Inventory Tracking:**  Allowing easy input and viewing of current stock levels for each candy, cake, and key ingredient. 
2. **Sales Reporting:** Generate reports showing which items are selling well and which are slower movers to inform production and promotional decisions.
3. **Ingredient Management:** Track inventory levels for common ingredients used in multiple recipes to prevent shortages.
4. **Expiration Date Reminders:**  Set expiration dates for perishable items and send alerts when they are approaching their expiry date to minimize waste.




INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"


Ironically the model with the most parameters (9bi instead of 7bi) was the one that least generated output:

```markdown
# Agent: consulting software engineer
## Final Answer: 
The MVP for Karoline's Kandy's inventory management system should focus on these core features:

1. **Simple Inventory Tracking:**  Allowing easy input and viewing of current stock levels for each candy, cake, and key ingredient. 
2. **Sales Reporting:** Generate reports showing which items are selling well and which are slower movers to inform production and promotional decisions.
3. **Ingredient Management:** Track inventory levels for common ingredients used in multiple recipes to prevent shortages.
4. **Expiration Date Reminders:**  Set expiration dates for perishable items and send alerts when they are approaching their expiry date to minimize waste.
```

`gemma2:9bi` output also seems more overengineering than others, missing the point that its Task was meant to be on a limited budget (I mean why care about inventory if you are literally just getting started with your small bakery?).

### Using a Manager (hierarchical process)

Moving on from the basics let's check out the Hierarchical process that's available out-of-the-box for CrewAi. This process mode aims to simulate traditional collaboration by adding in a "manager" and hierarchies between the agents.

Per the official docs:

> The hierarchical process in CrewAI introduces a structured approach to task management, simulating traditional organizational hierarchies for efficient task delegation and execution. This systematic workflow enhances project outcomes by ensuring tasks are handled with optimal efficiency and accuracy.

Before we move into this let's redefine our experiment's scope and convert our configuration into files so we may test out the API that's primary recommended by CrewAI Docs.

#### The CrewAI Yaml files

Each of the concepts we have used so far as parameters on `Python` code are also loadable from files using the YAML syntax, this allows for decoupling of code-logic from agent-specific logic and would, in theory, allow you to easily modify prompts and Tasks on a deployed system by simply changing the YAML files instead of deploying a new code change.

For our new experiment let's think about a team that works together to keep a social media account going. They should be aware of their handle (the @ username), a topic for a post and a set of tags that might be relevant to their audience.

The files themselves are on the [config](./config/) directory.

#### Loading the files into the environment

CrewAi.project provides us with annotations to allow reading from its `yaml` files and creating objects based on their attributes.

The base of this is the `@CrewBase` annotation that must be applied to a class, this annotation allows us to load the configs into class attributes and further use the other annotations (`@agent`, `@task`, `@crew`) to create functions that allow the Crew to work.

One cool thing on this approach is the usage o `{placeholders}` within the yaml file to allow for one to provide different inputs to the same crew without having to recreate the Tasks.

One hack I had to do, in order for this to work on the notebook, was to create an empty `__init__.py` file and to extract the Crew logic itself into the `crew.py` file due to same weird dependencies getting mixed up by the Jupyter kernel.

In [4]:
from crew import SocialMediaCrew

SocialMediaCrew().crew().kickoff(inputs = {
    'topic': 'como escolher um multiferramentas para levar em viagens de bicicleta',
    'language': 'pt-br',
    'brand': '@gravel.alves'
})

# outputs are stored under the ./outputs folder


Overriding of current TracerProvider is not allowed


CrewOutput(raw="- **Video Materials**\n   1. *Script* - The script should include all the important points from the context, such as selecting a quality and durable multi-tool, its essential tools (screwdrivers, wrenches, knife, keyring, oil brush, etc.), resistance to outdoor environment, inclusion of a first-aid kit, versatility for other uses, compactness, ease of use, research before buying, and the importance of designer Guino-M's multi-tool.\n   2. *Visuals* - The video should include shots of various multi-tools, their features, and close-ups of the essential tools. Include scenes of a multi-tool being used for maintenance tasks on a bike, outdoor environments to show its resistance, and scenes of other uses (car or electronic equipment).\n   3. *Voiceover* - The voiceover should be clear, concise, and engaging. It should explain the importance of each feature and tool, and emphasize the benefits of choosing Guino-M's multi-tool for bike trips.\n   4. *Music & Sound Effects* - U

### Using Knowledge

### Using Tools